In [1]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from utils import bootcampviztools as bt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
df = pd.read_csv("../data_sample/Absenteeism_at_work.csv", sep=";")

In [3]:
target = "Absenteeism time in hours"

In [4]:
df['Has_son'] = df['Son'].apply(lambda x: 1 if x > 0 else 0)
df.drop(["Son"], axis=1, inplace=True)

In [5]:
df['Has_pet'] = df['Pet'].apply(lambda x: 1 if x > 0 else 0)
df.drop(["Pet"], axis=1,inplace=True)

In [6]:
df.drop(columns=["Height", "Weight"], inplace=True)

In [7]:
X = df.drop([target], axis=1)
y = df[target]

In [8]:
y = (df['Absenteeism time in hours'] > df['Absenteeism time in hours'].median()).astype(int)

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:

from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from xgboost import XGBClassifier

In [11]:
lr_clf = LogisticRegression(max_iter=2000, class_weight="balanced")
rf_clf= RandomForestClassifier(max_depth=10, random_state=42, class_weight="balanced")
lgb_clf = LGBMClassifier(max_depth=10, random_state=42, verbose=-100, class_weight="balanced", n_jobs=-1)
xgb_clf = XGBClassifier(max_depth=10, random_state=42, n_jobs=-1)

In [12]:
print(f"RandomForest: {np.mean(cross_val_score(rf_clf, X_train, y_train, cv=5, scoring="balanced_accuracy"))}")
print(f"Logistic: {np.mean(cross_val_score(lr_clf, X_train, y_train, cv=5, scoring="balanced_accuracy"))}")
print(f"LGB: {np.mean(cross_val_score(lgb_clf, X_train, y_train, cv=5, scoring="balanced_accuracy"))}")
print(f"XGB: {np.mean(cross_val_score(xgb_clf, X_train, y_train, cv=5, scoring="balanced_accuracy"))}")

RandomForest: 0.7681207912457912


c:\Users\emmag\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\emmag\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/st

Logistic: 0.7574242424242424
LGB: 0.7773800505050505
XGB: 0.7698179713804714


In [13]:
logreg = LogisticRegression(max_iter=1000, random_state=42)
param_logreg = {
    'penalty': ['l1', 'l2', None],
    'solver': ['liblinear', 'saga'],  
    'C': [0.01, 0.1, 1, 10]
}
grid_logreg = GridSearchCV(logreg, param_logreg, cv=5, scoring='accuracy', n_jobs=-1)
grid_logreg.fit(X_train, y_train)

c:\Users\emmag\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
20 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\emmag\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\emmag\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\emmag\AppDa

GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=1000, random_state=42),
             n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1, 10], 'penalty': ['l1', 'l2', None],
                         'solver': ['liblinear', 'saga']},
             scoring='accuracy')

In [14]:
rf = RandomForestClassifier(random_state=42)
param_rf = {
    'n_estimators': [100, 200, 400],
    'max_depth': [None, 1, 5, 10],
    'min_samples_leaf': [1,10,20,100],
    'class_weight':['balanced', None],
    'max_features':['sqrt', 'log2', None]
}
grid_rf = GridSearchCV(rf, param_rf, cv=5, scoring='accuracy', n_jobs=-1)
grid_rf.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'class_weight': ['balanced', None],
                         'max_depth': [None, 1, 5, 10],
                         'max_features': ['sqrt', 'log2', None],
                         'min_samples_leaf': [1, 10, 20, 100],
                         'n_estimators': [100, 200, 400]},
             scoring='accuracy')

In [15]:
lgbm = LGBMClassifier(random_state=42) 
param_lgbm = {
    'n_estimators': [100, 200, 400],
    'learning_rate': [0.1, 0.3, 0.6, 1], 
    'max_depth': [1, 6, 10, -1],  
    'min_child_samples': [1, 10, 20, 100], 
    'scale_pos_weight': [
        len(df[df[target]==0]) / len(df[df[target]==1]),
        1],
    'colsample_bytree': [0.5, 1]
}
grid_lgbm = GridSearchCV(lgbm, param_lgbm, cv=5, scoring='accuracy', n_jobs=-1)
grid_lgbm.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=LGBMClassifier(random_state=42), n_jobs=-1,
             param_grid={'colsample_bytree': [0.5, 1],
                         'learning_rate': [0.1, 0.3, 0.6, 1],
                         'max_depth': [1, 6, 10, -1],
                         'min_child_samples': [1, 10, 20, 100],
                         'n_estimators': [100, 200, 400],
                         'scale_pos_weight': [0.5, 1]},
             scoring='accuracy')

In [16]:
xgb = XGBClassifier(max_depth=5, random_state=42)
param_xgb = {
    'n_estimators': [100, 200, 400],
    'eta': [0.1, 0.3, 0.6, 1],
    'max_depth': [1, 6, 10, None],
    'min_child_weight': [1,10,20,100],
    'colsample_bytree': [0.5,1]
}
grid_xgb = GridSearchCV(xgb, param_xgb, cv=5, scoring='accuracy', n_jobs=-1)
grid_xgb.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, feature_weights=None,
                                     gamma=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=Non...
                                     max_delta_step=None, max_depth=5,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.5, 1],
                         'eta': [0.1, 0.3, 0.6, 1],
                         'max_depth': [1, 6, 10, None],
                         'min_child_weight': [1, 10, 20, 100],
                         'n_estimators': [100, 200, 400]},
             scoring='accuracy')

In [17]:
from sklearn.metrics import accuracy_score

# Evaluación
models = {
    "LogisticR": grid_logreg,
    "RandomForest": grid_rf,
    "LightGBM": grid_lgbm,
    "XGBoost": grid_xgb
}

for name, model in models.items():
    acc = accuracy_score(y_test, model.predict(X_test))
    print(f"{name} - Best Params: {model.best_params_} | CV Score: {model.best_score_:.4f} | Test Accuracy: {acc:.4f}")

LogisticR - Best Params: {'C': 1, 'penalty': 'l1', 'solver': 'liblinear'} | CV Score: 0.7619 | Test Accuracy: 0.7365
RandomForest - Best Params: {'class_weight': 'balanced', 'max_depth': 5, 'max_features': None, 'min_samples_leaf': 1, 'n_estimators': 400} | CV Score: 0.7958 | Test Accuracy: 0.7568
LightGBM - Best Params: {'colsample_bytree': 0.5, 'learning_rate': 0.6, 'max_depth': 1, 'min_child_samples': 1, 'n_estimators': 200, 'scale_pos_weight': 1} | CV Score: 0.7974 | Test Accuracy: 0.8176
XGBoost - Best Params: {'colsample_bytree': 0.5, 'eta': 1, 'max_depth': 1, 'min_child_weight': 1, 'n_estimators': 100} | CV Score: 0.7872 | Test Accuracy: 0.8041


*LightGBM y XGBoost han obtenido el mejor rendimiento en test (81.8% y 80.4% de accuracy respectivamente). Para este dataset, LightGBM es la opción más robusta, combinando alta precisión con estabilidad en CV.*